In [3]:
###### Inputs ##########

##################################################
# Later, Specify the study/series/sop of the SEG you want
# as a list in the manifest file. 
##################################################

# Manifest - list of SeriesInstanceUIDs of the DICOM Segmentation objects 
# manifest_file = "C://Users//deepa//git//nlst_explore//manifest//manifest_list.txt"
# gcs_file = "C://Users//deepa//git//nlst_explore//manifest//gcs_list.txt" 

# GCP settings 
project_id = "idc-external-025"
dataset_id = "total_segmentator_nlst_total_070823"
datastore_id = "126k-series"
location_id = "us-central1"

# table 
table_id = "bigquery-public-datasets.idc_current.dicom_all"

# specify for now - the seg series
studyInstanceUID = "1.2.840.113654.2.55.222358418596811829743750475747304078456"
seriesInstanceUID ="1.2.276.0.7230010.3.1.3.313263360.50439.1688804452.744648" 
sopInstanceUID = "1.2.276.0.7230010.3.1.4.313263360.50439.1688804452.744649"

# Not sure if this needs to be set 
%env CLOUDSDK_PYTHON=C:\Users\deepa\AppData\Local\slicer.org\Slicer 5.5.0-2023-08-19\bin\PythonSlicer.exe


env: CLOUDSDK_PYTHON=C:\Users\deepa\AppData\Local\slicer.org\Slicer 5.5.0-2023-08-19\bin\PythonSlicer.exe


In [4]:
### Imports and install packages ### 

import os 
import sys 
from DICOMLib import DICOMPlugin
import vtk, qt, ctk, slicer

import DicomRtImportExportPlugin
import DICOMwebBrowser
from DICOMwebBrowser import GoogleCloudPlatform

import dicomweb_client.log
dicomweb_client.log.configure_logging(2)
from dicomweb_client.api import DICOMwebClient

from DICOMLib import DICOMUtils

import pydicom 

from slicer.util import pip_install
pip_install("google-cloud")
pip_install("google-cloud-bigquery")
pip_install("google-cloud-storage")
pip_install("db-dtypes")

!pip install google-cloud-bigquery-storage 
!pip install pyarrow
!pip install db-dtypes

!pip install --upgrade pydata-google-auth google-cloud-bigquery[pandas]

from google.cloud import bigquery
from google.cloud import storage
import db_dtypes

# Setup gcloud 
import shutil 
args = [shutil.which('gcloud')]
if (None in args):
   logging.error(f"Unable to locate gcloud, please install the Google Cloud SDK")
else: 
    print ("found gcloud")

import pydata_google_auth
credentials = pydata_google_auth.get_user_credentials(
    ['https://www.googleapis.com/auth/bigquery'],
)



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python-real.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python-real.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python-real.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python-real.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Users\deepa\miniconda3\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Users\deepa\miniconda3\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Users\deepa\miniconda3\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Users\deepa\miniconda3\python.exe -m pip install --upgrade pip


found gcloud


In [5]:
########################################################################
#-- Download the DICOM segmentation objects from the DICOM datastore --#
########################################################################

# Setup the DICOMwebClient 
url = f"https://healthcare.googleapis.com/v1/projects/{project_id}/locations/{location_id}/datasets/{dataset_id}/dicomStores/{datastore_id}/dicomWeb"
session = None
headers = {}
headers["Authorization"] = f"Bearer {GoogleCloudPlatform().token()}"
DICOMwebClient = DICOMwebClient(url=url, session=session, headers=headers)

# first set the downLoad directory 
downloadDirectory_seg = os.path.join(slicer.dicomDatabase.databaseDirectory, 'tmp_seg')
if not os.path.isdir(downloadDirectory_seg):
  os.makedirs(downloadDirectory_seg)

# Retrieve the instance using the DICOM web client  
retrievedInstance = DICOMwebClient.retrieve_instance(study_instance_uid=studyInstanceUID,
                                                     series_instance_uid=seriesInstanceUID, 
                                                     sop_instance_uid=sopInstanceUID)

# Write the file to a tmp folder 
pydicom.filewriter.write_file(os.path.join(downloadDirectory_seg, "seg.dcm"), retrievedInstance)

# Import file into the dicom database 
db = slicer.dicomDatabase
DICOMUtils.importDicom(downloadDirectory_seg,db)

2023-09-15 17:18:33,688 | INFO     | dicomweb_client.web                      | retrieve instance "1.2.276.0.7230010.3.1.4.313263360.50439.1688804452.744649" of series "1.2.276.0.7230010.3.1.3.313263360.50439.1688804452.744648" of study "1.2.840.113654.2.55.222358418596811829743750475747304078456"


True

In [7]:
# Get the referenced SeriesInstanceUID

dcm = pydicom.dcmread(os.path.join(downloadDirectory_seg, "seg.dcm"))
referenced_series_instance_uid =  dcm.ReferencedSeriesSequence[0]['SeriesInstanceUID'].value
print('referenced_series_instance_uid: ' + str(referenced_series_instance_uid))

referenced_series_instance_uid: 1.2.840.113654.2.55.214507427258856342030051653305995868983


In [8]:
# Select all files from GCS for a given DICOM series

client = bigquery.Client(project=project_id)

query = f"""
SELECT 
  # DISTINCT(CONCAT("cp s3://", SPLIT(gcs_url,"/")[SAFE_OFFSET(2)], "/", crdc_series_uuid, "/* .")) 
  # gcs_url
  DISTINCT(crdc_series_uuid) as series_uuid 
FROM 
    `bigquery-public-data.idc_current.dicom_all`
WHERE 
  SeriesInstanceUID = @series_id;
"""

job_config = bigquery.QueryJobConfig(query_parameters=[
                                                       bigquery.ScalarQueryParameter("series_id", "STRING", referenced_series_instance_uid)
                                                       ])
result = client.query(query, job_config=job_config) 
series_df = result.to_dataframe(create_bqstorage_client=True)
print(series_df)

                            series_uuid
0  edfd3f31-6e67-4f0d-815f-c37a5bc022c3


In [9]:
# first set the downLoad directory 

downloadDirectory = os.path.join(slicer.dicomDatabase.databaseDirectory, 'tmp')
if not os.path.isdir(downloadDirectory):
  os.makedirs(downloadDirectory)
    

In [10]:
# Use gsutil to copy the files to the downloadDirectory 

gcs_directory = 'gs://public-datasets-idc/' + series_df['series_uuid'].values[0]
print('gcs_directory: ' + gcs_directory)
!gsutil -m cp -r $gcs_directory/* $downloadDirectory


gcs_directory: gs://public-datasets-idc/edfd3f31-6e67-4f0d-815f-c37a5bc022c3


Copying gs://public-datasets-idc/edfd3f31-6e67-4f0d-815f-c37a5bc022c3/0a4dabf1-92c3-4e1c-8704-b0fe885a6391.dcm...
/ [0/96 files][    0.0 B/ 48.2 MiB]   0% Done                                   
Copying gs://public-datasets-idc/edfd3f31-6e67-4f0d-815f-c37a5bc022c3/08cff637-dc9e-4202-b7f1-543dcd20d913.dcm...
/ [0/96 files][    0.0 B/ 48.2 MiB]   0% Done                                   
Copying gs://public-datasets-idc/edfd3f31-6e67-4f0d-815f-c37a5bc022c3/156ed413-706c-40f0-9664-f6a665d0ea21.dcm...
Copying gs://public-datasets-idc/edfd3f31-6e67-4f0d-815f-c37a5bc022c3/2264e182-9031-4dfa-a0e7-520bce9401e3.dcm...
/ [0/96 files][    0.0 B/ 48.2 MiB]   0% Done                                   
Copying gs://public-datasets-idc/edfd3f31-6e67-4f0d-815f-c37a5bc022c3/0e16df6a-700b-4cb0-a6d1-97c0ab745d90.dcm...
/ [0/96 files][    0.0 B/ 48.2 MiB]   0% Done                                   
Copying gs://public-datasets-idc/edfd3f31-6e67-4f0d-815f-c37a5bc022c3/10b89525-d226-4d21-81d8-d6fe10760ce9

In [11]:
# Now add the CT files to the DICOM database 
    
DICOMUtils.importDicom(downloadDirectory,db)


True

In [12]:
#-- Load the CT DICOM files into the scene --# 

loadedNodeIDs = []  # this list will contain the list of all loaded node IDs

patientUIDs = db.patients()
for patientUID in patientUIDs:
    loadedNodeIDs.extend(DICOMUtils.loadPatientByUID(patientUID))

In [ ]:
#-- Load the DICOM segmentation file into the scene --# 

# DICOMSegmentationPlugin = slicer.modules.dicomPlugins['DICOMSegmentationPlugin']()
# # loadables = DICOMSegmentationPlugin.examineFiles([fileName])
# loadables = DICOMSegmentationPlugin.examineFiles([os.path.join(downloadDirectory_seg,'seg.dcm'])
# DICOMSegmentationPlugin.load(loadables[0])


In [13]:
# #-- Get the reference volume node --#
# # The loadedNodeIDs will most likely contain a single scalar volume node, but if not, search for it 

# referenceVolumeNode = [] 
# for nodeID in loadedNodeIDs: 
#   node = slicer.mrmlScene.GetNodeByID(nodeID)
#   if node.IsA('vtkMRMLScalarVolumeNode'):
#     referenceVolumeNode = node 
# if not referenceVolumeNode: 
#   print('ERROR creating scalar volume from DICOM data directory, please check your data')

<class 'AttributeError'>: 'NoneType' object has no attribute 'IsA'

In [ ]:
# #-- Create the subject hierarchy --# 

# # First get the list of segmentationNodes 
# segmentationNodes = slicer.util.getNodes('*SegmentationNode*') # segmentationNodes = slicer.util.getNodesByClass('*Segmentation*')
# segmentationNodes = list(segmentationNodes.values())

# # Get the subject hierarchy 
# shNode = slicer.mrmlScene.GetSubjectHierarchyNode()
# seriesItem = shNode.GetItemByDataNode(referenceVolumeNode)
# studyItem = shNode.GetItemParent(seriesItem)
# patientItem = shNode.GetItemParent(studyItem)

# # Set the parent for each of the segmentationNodes 
# for segmentationNode in segmentationNodes: 
#   segmentationShItem = shNode.GetItemByDataNode(segmentationNode)
#   shNode.SetItemParent(segmentationShItem, studyItem)

